## Data Exploration

In this notebook, we will use Python to better understand the data available

In [27]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [28]:
# Define the path to the data based on input type
def set_csv(input_type):
    valid_types = {
        "data-set-1": "../raw-data/fraud-data-1.csv",
        "data-set-2": "../raw-data/fraud-data-2.csv",
    }
    
    if input_type not in valid_types:
        raise ValueError(f"Invalid type '{input_type}'. Expected one of {list(valid_types.keys())}")
    
    return valid_types[input_type]

In [29]:
# DO THIS: Change csv_name input to data-set-1 or data-set-2
try:
    csv_name = "data-set-2" 
    df = pd.read_csv(set_csv(csv_name))
    # confirm the file we're looking at
    print(f"Successfully read {csv_name.upper()} CSV file. Continuing with analysis...")
except ValueError as e:
    print(e)

Successfully read DATA-SET-2 CSV file. Continuing with analysis...


In [30]:
# Show summary of the dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 17 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   order_id                               500 non-null    int64  
 1   user_id                                500 non-null    object 
 2   order_date                             500 non-null    object 
 3   booking_variety                        500 non-null    object 
 4   hotel_country                          216 non-null    object 
 5   booking_days_before_checkin/departure  500 non-null    int64  
 6   flight_type                            238 non-null    object 
 7   flight_departure_country               238 non-null    object 
 8   flight_arrival_country                 238 non-null    object 
 9   amount_usd                             500 non-null    float64
 10  payment_method                         500 non-null    object 
 11  phone 

In [31]:
# display the first 10 rows of the dataframe
df.head(10)

,order_id,user_id,order_date,booking_variety,hotel_country,booking_days_before_checkin/departure,flight_type,flight_departure_country,flight_arrival_country,amount_usd,payment_method,phone number,issuer_country,billing_country,ip_country,number_guest,ancillaries/referrals
0,889029710,006a-70a7-4db2-a3b5-2470,2023-06-18,Flight,NaN,8,return,CA,CA,444.80,card,161152214511,CA,CA,CA,1,PF
1,762488558,81e4-d09c-4rtc-8g76-7cdf0,2023-10-13,Flight,NaN,1,one_way,US,BR,1124.54,card,13218815374,US,US,MX,2,NaN
2,997264060,0323-cf9d-4628-8db2-7cdb,2023-06-26,Flight,NaN,11,return,CA,US,163.97,card,1247982008,CA,CA,CA,1,NaN
3,570146346,620a-945b-4213-b900-4f7e,2023-10-14,Flight,NaN,1,one_way,US,MX,1154.54,card,132151818374,US,US,MX,2,NaN
4,868653733,03cc-b95f-4d18-9ab4-c712,2023-08-24,Flight,NaN,12,return,US,US,142.03,card,1479326522,US,US,US,2,NaN
5,319226153,03d8-7c34-499d-b1f7-fde3,2023-02-19,Flight,NaN,5,one_way,US,US,144.53,google_pay,1937459001,US,US,US,1,NaN
6,168455081,03fd-5291-471d-95b7-d6b3,2023-03-29,Hotel,US,10,NaN,NaN,NaN,130.52,card,15758541345,US,US,US,2,CFAR
7,125067599,0452-9abc-4a9f-acc9-eb47,2023-07-02,Flight,NaN,12,return,US,US,127.87,card,1305239671,US,US,US,1,PF
8,807005738,0477-a158-4e55-9e32-1ed2,2023-06-25,Hotel,US,8,NaN,NaN,NaN,54.21,card,1443079572,US,US,US,1,NaN
9,527145986,047d-f44e-4a41-aac0-f292,2023-10-10,Flight,NaN,11,one_way,DO,US,1484.57,card,19195511536,US,US,US,2,PF


In [32]:
# Find how many unique values are in each column
df.nunique()

order_id                                 500
user_id                                  500
order_date                               232
booking_variety                            3
hotel_country                             14
booking_days_before_checkin/departure     29
flight_type                                2
flight_departure_country                  15
flight_arrival_country                    18
amount_usd                               453
payment_method                             5
phone number                             479
issuer_country                            14
billing_country                           14
ip_country                                15
number_guest                               5
ancillaries/referrals                      3
dtype: int64

In [33]:
# Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# Convert dates
df['order_date'] = pd.to_datetime(df['order_date'])

In [34]:
if csv_name == "data-set-1":
    # Flag false positives (flagged as fraud but approved with no chargeback)
    df['false_positive'] = ((df['flagged_as_fraud'] == 'Yes') &
                            (df['result'] == 'Approved') &
                            (df['chargeback_(yes/no)'] == 'No'))

    # Flag false negatives (not flagged as fraud but resulted in chargeback)
    df['false_negative'] = ((df['flagged_as_fraud'] == 'No') &
                            (df['chargeback_(yes/no)'] == 'Yes'))

    print("False Positives:", df['false_positive'].sum())
    print("False Negatives:", df['false_negative'].sum())

    # Chargebacks by payment method
    payment_type = df[df['chargeback_(yes/no)'] == 'Yes']['payment_method'].value_counts()
    print("Payment method chargeback count:", payment_type)

    # Average fraud score by result type
    average_score = df.groupby('result')['fraud_score'].mean()
    print("Average fraud score by result:", average_score)

    # Which agents had the most chargebacks
    agents = df[df['chargeback_(yes/no)'] == 'Yes']['agent_id'].value_counts()
    print("Agent chargeback count:", agents)

    sns.boxplot(data=df, x='result', y='fraud_score')
    plt.title('Fraud Score Distribution by Result')
    plt.show()

In [35]:
if csv_name == "data-set-2":

    # 1. Mismatched country information
    df['mismatched_countries'] = (
        (df['issuer_country'] != df['billing_country']) |
        (df['issuer_country'] != df['ip_country']) |
        (df['billing_country'] != df['ip_country'])
    )

    # 2. Last-minute booking (1 day or less)
    df['last_minute_booking'] = df['booking_days_before_checkin/departure'].astype(float) <= 1

    # 3. High value transactions (top 5%)
    high_value_threshold = df['amount_usd'].astype(float).quantile(0.95)
    df['high_value'] = df['amount_usd'].astype(float) > high_value_threshold

    # 4. International one-way flights
    df['intl_one_way'] = (
        (df['flight_type'] == 'one_way') &
        (df['flight_departure_country'] != df['flight_arrival_country'])
    )

    # 5. Combined fraud risk flag (if 2 or more conditions are met)
    df['potential_fraud'] = (
        df[['mismatched_countries', 'last_minute_booking', 'high_value', 'intl_one_way']]
        .sum(axis=1) > 1
    )

    # Summary outputs
    print("Mismatched countries:", df['mismatched_countries'].sum())
    print("Last-minute bookings:", df['last_minute_booking'].sum())
    print("High-value transactions:", df['high_value'].sum())
    print("International one-way flights:", df['intl_one_way'].sum())
    print("High-risk transactions (2+ signals):", df['potential_fraud'].sum())


Mismatched countries: 127
Last-minute bookings: 109
High-value transactions: 25
International one-way flights: 93
High-risk transactions (2+ signals): 117
